In [ ]:
print('Setup complete.')

# LAB 22 — SQL data explainer

```


## Original CLI usage

```bash
bash
python lessons/day-08/labs/lab-22/main.py query --db ./data.duckdb --q "top 10 repos by stars"
```


In [ ]:
# Environment check (adjust as needed)
import sys, platform
print(f'Python: {sys.version.split()[0]} on {platform.system()}')
# If imports fail, install requirements in your environment.
try:
    import httpx, jinja2, rich, typer, duckdb, pandas, matplotlib  # may be subset per lab
except Exception as e:
    print('Optional imports missing for some labs:', e)


In [ ]:
#!/usr/bin/env python3
"""LAB 22 — SQL data explainer

Minimal, safe demo using DuckDB over `data/sample_repos.csv`.
Supports a few preset NL queries -> SQL mappings and renders a bar chart.
"""
from __future__ import annotations

from pathlib import Path
from typing import Dict, Tuple

import duckdb
import typer
from rich import print
import matplotlib

# Use non-GUI backend for headless environments (e.g., CI)
matplotlib.use("Agg")
import matplotlib.pyplot as plt  # noqa: E402




SAMPLE_CSV = Path(__file__).resolve().parents[2] / "data" / "sample_repos.csv"
DEFAULT_DB = Path(__file__).resolve().parents[1] / "lab-22" / "data.duckdb"
DEFAULT_PNG = Path(__file__).resolve().parents[1] / "lab-22" / "output.png"


PRESETS: Dict[str, Tuple[str, str, str]] = {
    # name: (description, SQL, chart_kind)
    "top10_by_stars": (
        "Top 10 repos by stars",
        """
        SELECT name, stars
        FROM repos
        ORDER BY stars DESC
        LIMIT 10
        """,
        "bar",
    ),
    "count_by_language": (
        "Repo count by language",
        """
        SELECT language, COUNT(*) AS count
        FROM repos
        GROUP BY language
        ORDER BY count DESC
        """,
        "bar",
    ),
    "avg_stars_by_language": (
        "Average stars by language",
        """
        SELECT language, AVG(stars)::DOUBLE AS avg_stars
        FROM repos
        GROUP BY language
        ORDER BY avg_stars DESC
        """,
        "bar",
    ),
}


def ensure_db(db_path: Path) -> None:
    """Create DuckDB at `db_path` and load sample CSV into table `repos` if missing."""
    db_path.parent.mkdir(parents=True, exist_ok=True)
    init_needed = not db_path.exists()
    con = duckdb.connect(str(db_path))
    try:
        if init_needed:
            if not SAMPLE_CSV.exists():
                raise FileNotFoundError(f"Sample CSV not found: {SAMPLE_CSV}")
            con.execute(
                """
                CREATE TABLE repos AS
                SELECT * FROM read_csv_auto(?);
                """,
                [str(SAMPLE_CSV)],
            )
        else:
            # Validate table exists; if not, (re)create
            try:
                con.execute("SELECT 1 FROM repos LIMIT 1").fetchall()
            except duckdb.CatalogException:
                con.execute(
                    """
                    CREATE TABLE repos AS
                    SELECT * FROM read_csv_auto(?);
                    """,
                    [str(SAMPLE_CSV)],
                )
    finally:
        con.close()


def list_presets() -> None:
    """List available NL->SQL presets."""
    print("[bold]Available presets:[/]")
    for key, (desc, _, _) in PRESETS.items():
        print(f"- {key}: {desc}")


def query(
    preset: str = "top10_by_stars")}"
    ),
    db: Path = DEFAULT_DB,
    out: Path = DEFAULT_PNG,
) -> None:
    """Run a preset query and save a bar chart to PNG."""
    if preset not in PRESETS:
        raise typer.BadParameter(f"Unknown preset: {preset}")

    ensure_db(db)
    desc, sql, chart = PRESETS[preset]
    print(f"[cyan]Running:[/] {desc}")

    con = duckdb.connect(str(db))
    try:
        df = con.execute(sql).df()
    finally:
        con.close()

    if df.empty:
        print("[yellow]No data returned.[/]")
        return

    # Basic plotting logic
    plt.clf()
    if chart == "bar":
        # heuristic: use first column as x, last as y
        x = df.columns[0]
        y = df.columns[-1]
        df.plot(kind="bar", x=x, y=y, legend=False)
        plt.title(desc)
        plt.tight_layout()
        out.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out)
        print(f"[green]Chart saved to:[/] {out}")
    else:
        print("[yellow]Chart type not implemented.")

    # Print textual summary
    print("[bold]Result (head):[/]")
    print(df.head().to_string(index=False))


In [ ]:
# Try the functions defined above. For example, if this lab had commands like `draft()` or `apply()`:
# draft()
# apply()
